In [5]:
from statsmodels.regression.rolling import RollingOLS
import eikon as ek
ek.set_app_key('85053106977c410aae4a51a1299cebee57f2d209')
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')

"""
find_atr takes input [stock_data] and returns the normalized atr or average 
true range; atr decomposes range during our period to access volatility
"""
def find_atr(stock_data):
  atr = ta.atr(high=stock_data['high'], 
              low=stock_data['low'],
              close=stock_data['close'])
  return atr.sub(atr.mean()).div(atr.std())

"""
find_macd returns moving average convergence/divergence
"""
def find_macd(close):
  macd = ta.macd(close=close, length=20).iloc[:,0]
  return macd.sub(macd.mean()).div(macd.std())


"find_sma returns simple moving average"
def find_sma(close):
  sma = ta.sma(close=close, length=20)
  return sma.sub(sma.mean()).div(sma.std())


'''
Part 1: Data Cleaning
1. using eikon api, get s&p500 company names
2. set an 8 year window of data to examine
3. filter any dots out with dashs to avoid errors
4. create a list of all s&p 500 tickers
5. use yfinance to generate constituent data
6. clean constituent data
'''
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
start = '2016-02-24'
end = '2024-02-24'
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
sp500_tickers = sp500['Symbol'].unique().tolist()
data = yf.download(tickers= sp500_tickers, start=start, end=end).stack()
data.index.names = ['date', 'ticker']
data.columns = data.columns.str.lower()



"""
Part 2: Indicators
1. Garman-Klass Volatility
2. RSI
3. Bollinger Bands
4. ATR
5. MACD
6. Dollar Volume
7. SMA
"""
# GKV = (log(high)- log(low)^2)/2 - (2log(2) -1)(log(adj close) - log(open))^2
data['gkv'] = ((np.log(data['high']) - np.log(data['low']))**2)/2
- (2*np.log(2)-1) * ((np.log(data['adj close']) - np.log(data['open'])**2 ))
data['rsi'] = data.groupby(level=1)['adj close'].transform(lambda x: ta.rsi(close=x, length=20))
data['bb_low'] = data.groupby(level=1)['adj close'].transform(lambda x: ta.bbands(close=np.log1p(x),length=20).iloc[:,0])
data['bb_mid'] = data.groupby(level=1)['adj close'].transform(lambda x: ta.bbands(close=np.log1p(x),
length=20).iloc[:,1])
data['bb_high'] = data.groupby(level=1)['adj close'].transform(lambda x: ta.bbands(close=np.log1p(x),
length=20).iloc[:,2])
data['atr'] = data.groupby(level=1, group_keys=False).apply(find_atr)
data['macd'] = data.groupby(level=1, group_keys=False)['adj close'].apply(find_macd)
data['dv'] = (data['adj close']*data['volume'])/1e6
data['sma'] = data.groupby(level=1, group_keys=False)['adj close'].apply(find_sma)


"""
Part 3: Aggregate monthly level and filter for top 100 most liquid stocks for
each month
"""
last_cols = [c for c in data.columns.unique(0) if c not in ['dv', 'volume', 
                                                            'open','high',
                                                            'low', 'close']]
aggregate = pd.concat([data.unstack('ticker')['dv'].resample('M').mean().stack('ticker').to_frame('dv'),
data.unstack()[last_cols].resample('M').last().stack('ticker')],axis=1).dropna()


#find 5-year rolling average of dv to filter
aggregate['dv'] = (aggregate['dv'].unstack('ticker').rolling(5*12).mean().stack())
aggregate['dv_rank'] = (aggregate.groupby('date')['dv'].rank(ascending=False))

#isolate top 100
aggregate[aggregate['dv_rank']<100].drop(['dv', 'dv_rank'], axis=1)

"""
Part 4: Find monthly returns for different time horizons as features
"""

"returns 3 month returns for each ticker while clipping outliers at 5% threshold"
def get_returns(data):
  outlier = 0.05
  lags = [1,3,6,9,12]
  for lag in lags:
    data[f'return_{lag}m'] = (data['adj close']
                           .pct_change(lag)
                           .pipe(lambda x: x.clip(lower=x.quantile(outlier),
                                                  upper= x.quantile(1-outlier)))
                           .add(1)
                           .pow(1/lag)
                           .sub(1)
                           )
    return data
  
  aggregate = aggregate.groupby(level=1, group_keys=False).apply(get_returns).dropna()


  """
  Part 5:
  Download Fama-French Factors and Calculate Rolling Factor Betas
  """
  ff_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                 'famafrench',
                 start = '2010')[0]

  ff_data.index = ff_data.index.to_timestamp()

  print(ff_data)







[*********************100%%**********************]  503 of 503 completed
